In [ ]:
import os

import pandas as pd
import streamlit as st

In [ ]:
header = st.container()
dataset = st.container()

In [ ]:
with header:
    st.title("Premiere Movie Cater Just for You")

In [ ]:
with dataset:
    df = pd.read_csv(os.path.join(os.curdir, "data", "ml-latest-small", "movies.csv"))
    movie = st.selectbox(
        label="Select a Movie",
        options=df["title"],
    )
    df.query("title == @movie")["movieId"].iloc[0]